# 세그먼트 용 데이터 준비 EDA

상품 구매 정보-제휴사 이용-엘페이 이용 정보를 조인하여 보는게 불가능한 현상 인식

따라서 각 테이블을 독립 분석하여 사용하고자 함

# 준비

- 나눔고딕 폰트 설치
- 구글 드라이브 연결
- 데이터 로딩

In [1]:
import pandas as pd
import os
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
# plt.rcParams['font.family'] = 'NanumGothic'
sns.set(font="NanumGothic", 
        rc={"axes.unicode_minus":False, 'figure.figsize': (10,6)},
        style='darkgrid')
# sns.set_theme(style="whitegrid")

In [3]:
file_path_dic = {
    'demo':'LPOINT_BIG_COMP_01_DEMO.csv',
    'pdde':'LPOINT_BIG_COMP_02_PDDE.csv',
    'cop_u':'LPOINT_BIG_COMP_03_COP_U.csv',
    'pd_clac':'LPOINT_BIG_COMP_04_PD_CLAC.csv',
    'br':'LPOINT_BIG_COMP_05_BR.csv',
    'lpay':'LPOINT_BIG_COMP_06_LPAY.csv',
}

In [4]:
base_path = r'LPOINT_BIG_COMP_csv파일'

## demo: 고객 데모 정보

In [5]:
df_demo = pd.read_csv(os.path.join(base_path, file_path_dic['demo']))

## 상품 분류 정보: 유통사 상품 카테고리 마스터

In [6]:
df_pd_clac = pd.read_csv(os.path.join(base_path, file_path_dic['pd_clac']))

In [7]:
df_pd_clac.describe()

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
count,1933,1933,1933,1933
unique,1933,1932,60,349
top,PD0001,기타,패션잡화,어류
freq,1,2,130,20


## 점포 정보: 유통사/제휴사 점포 마스터

In [8]:
df_br = pd.read_csv(os.path.join(base_path, file_path_dic['br']))

## 상품 구매 정보: 유통사 상품 구매 내역

중복행 제거
- 원본 테이블: 4,381,743 건
- 중복건수: 237,354 건
- 중복제거후: 4,144,389 건
    
buy_ct 추가
- de_dt와 de_hr 연결하야 timestamp 생성

In [9]:
df_pdde_raw = pd.read_csv(os.path.join(base_path, file_path_dic['pdde']), dtype={'br_c':str, 'de_dt':str, 'de_hr':str, 'buy_am':np.int64})

In [10]:
print(df_pdde_raw.shape, df_pdde_raw.duplicated().sum())

(4381743, 10) 237354


In [11]:
df_pdde_raw.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000,1
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700,1
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000,1
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000,1
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000,1


### 장바구니 단순 특성 조사

In [12]:
gr_rct_no = df_pdde_raw.groupby(['rct_no'])

In [13]:
df_pdde_raw['de_dthr'] = df_pdde_raw.de_dt + df_pdde_raw.de_hr

In [14]:
print("장바구니에 담긴 cop_c가 2개이상인 경우의 갯수", (gr_rct_no.cop_c.nunique() > 1).sum())

장바구니에 담긴 cop_c가 2개이상인 경우의 갯수 0


In [15]:
print("장바구니에 담긴 br_c가 2개이상인 경우의 갯수", (gr_rct_no.br_c.nunique() > 1).sum())

장바구니에 담긴 br_c가 2개이상인 경우의 갯수 0


In [16]:
print("장바구니에 담긴 chnl_dv가 2개이상인 경우의 갯수", (gr_rct_no.chnl_dv.nunique() > 1).sum())

장바구니에 담긴 chnl_dv가 2개이상인 경우의 갯수 0


In [17]:
print("장바구니에 담긴 de_dthr가 2개이상인 경우의 갯수", (gr_rct_no.de_dthr.nunique() > 1).sum())

장바구니에 담긴 de_dthr가 2개이상인 경우의 갯수 0


In [18]:
def show_simple_ecdf_from_value_counts(ss, x_label='cnts', y_label='prob', kind='scatter', title='', logx=False,):
    y = (ss.cumsum() / ss.sum()).rename(y_label)
    pd.DataFrame(y.rename_axis(x_label).reset_index()).plot(kind=kind, title=title, x=x_label, y=y_label, ylim=[0,1], logx=logx)
    print(f'Most frequent case is {y.index[0]} with pct {y.iloc[0]*100:.2f}%')

#### 사용자 관점 트랜잭션/결제 횟수 분포

사용자별 결제건수
- 중간값: 30번
- 평균값: 47번

사용자별 총결제금액
- 중간값: 1.4 백만원
- 평균값: 3.8 백만원


In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.cust.value_counts().value_counts().sort_index(), title='사용자별 트랜잭션수 분포')

In [ ]:
pd.DataFrame(df_pdde_raw.groupby('cust').rct_no.nunique().describe()).T

In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.groupby('cust').rct_no.nunique().value_counts().sort_index(), title='사용자별 결제 횟수 분포')

In [ ]:
pd.DataFrame(df_pdde_raw.groupby('cust').buy_am.sum().describe()).T

In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.groupby('cust').buy_am.sum().value_counts().sort_index(), x_label='결제금액', title='사용자별 결제 금액 분포', logx=True)

#### 장바구니에 담긴 pd_c 를 조사

- 다양성
    - 약 60%는 1가지만
    - 약 75가지 담기는 경우도 존재
- 트랜잭션 수
    - 약 50%는 1건
    - 최대 120건도 존재

In [ ]:
show_simple_ecdf_from_value_counts(gr_rct_no.pd_c.nunique().value_counts().sort_index(), title='장바구니별 pd_c 가짓수 분포')

In [ ]:
show_simple_ecdf_from_value_counts(gr_rct_no.pd_c.count().value_counts().sort_index(), title='장바구니별 pd_c 카운트 분포')

#### [rct_no, pd_c] 를 조사

- buy_am의 다양성
    - 약 90%는 1가지
    - 약 19가지도 존재
- buy_ct의 다양성
    - 약 95%는 1가지
    - 8가지도 존재

In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.groupby(['rct_no', 'pd_c']).buy_am.nunique().value_counts().sort_index(), title='장바구니-pd_c별 구매금액 가짓수 분포')

In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.groupby(['rct_no', 'pd_c']).buy_ct.nunique().value_counts().sort_index(), title='장바구니-pd_c별 상품수량 가짓수 분포')

In [ ]:
show_simple_ecdf_from_value_counts(df_pdde_raw.groupby(['rct_no', 'pd_c']).buy_ct.count().value_counts().sort_index(), title='장바구니-pd_c별 상품수량 카운트 분포')

In [19]:
df_pdde_raw['cust_uniq_rct_nos'] = df_pdde_raw.groupby('cust').rct_no.transform('nunique')

In [20]:
df_pdde_raw['cust_total_payments'] = df_pdde_raw.groupby('cust').buy_am.transform('sum')

In [21]:
df_pdde_raw['rct_no_uniq_pd_cs'] = df_pdde_raw.groupby(['rct_no']).pd_c.transform('nunique')

In [22]:
df_pdde_raw['rct_no_pd_cs'] = df_pdde_raw.groupby(['rct_no']).pd_c.transform('count')

In [23]:
df_pdde_raw['rct_no_payment'] = df_pdde_raw.groupby(['rct_no']).buy_am.transform('sum')

In [24]:
df_pdde_raw['de_dthr'] = pd.to_datetime(df_pdde_raw.de_dt.str.cat(df_pdde_raw.de_hr), format='%Y%m%d%H')

In [25]:
df_pdde_raw.drop(labels=['de_dt', 'de_hr'], axis=1, inplace=True)

In [26]:
df_pdde_raw.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,buy_am,buy_ct,de_dthr,cust_uniq_rct_nos,cust_total_payments,rct_no_uniq_pd_cs,rct_no_pd_cs,rct_no_payment
0,M430112881,A01000001113,1,A01,A010039,PD0290,15000,1,2021-01-01 10:00:00,102,2977947,1,1,15000
1,M646853852,A01000002265,1,A01,A010025,PD1369,79700,1,2021-01-01 10:00:00,97,9165313,1,1,79700
2,M430112881,A01000003148,1,A01,A010039,PD0290,19000,1,2021-01-01 10:00:00,102,2977947,1,2,38000
3,M430112881,A01000003148,1,A01,A010039,PD0290,19000,1,2021-01-01 10:00:00,102,2977947,1,2,38000
4,M430112881,A01000004946,1,A01,A010039,PD0290,19000,1,2021-01-01 10:00:00,102,2977947,1,1,19000


---

# RFM 세그먼트

## 마진 설정

In [27]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.00)    
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

## RFM

In [28]:
# 장바구니별로 groupby
df_crm = (df_pdde_raw.groupby(['rct_no'])
         .agg({'cust':'first', 'de_dthr':'first', 'rct_no_payment':'first'}) # cust_uniq_rct_nos':'first'
         .sort_values(by='de_dthr', ascending=True)
         .reset_index() # drop=True
         .rename(columns={'rct_no_payment':'buy_am'})   # 'cust_uniq_rct_nos':'rct_cnt'
)
df_crm

,rct_no,cust,de_dthr,buy_am
0,E02000052541,M978764630,2021-01-01 00:00:00,444800
1,E02000057891,M601576068,2021-01-01 00:00:00,40290
2,E02000056622,M726383544,2021-01-01 00:00:00,41230
3,E02000055173,M102283139,2021-01-01 00:00:00,9360
4,E02000054678,M564299660,2021-01-01 00:00:00,30500
...,...,...,...,...
1266584,A04128987327,M562823758,2021-12-31 23:00:00,9000
1266585,A04128847016,M326961541,2021-12-31 23:00:00,31150
1266586,A04128958050,M629230802,2021-12-31 23:00:00,2000
1266587,E03012401343,M655525572,2021-12-31 23:00:00,60080


## 상반기 하반기 데이터 나누기

In [29]:
# 상반기는 2350177건
# 하반기는 2031629건
half = '2021-07-01'
# second_half = '2021-12-31'

# 상반기 데이터: df_first_half, 하반기 데이터: df_second_half
df_first_half = df_pdde_raw.query('de_dthr < @half ')
df_second_half = df_pdde_raw.query('de_dthr >= @half')

### 상반기

In [30]:
# 상반기 장바구니별로 groupby
df_crm_first = (df_first_half.groupby(['rct_no'])
         .agg({'cust':'first', 'de_dthr':'first', 'rct_no_payment':'first'}) # cust_uniq_rct_nos':'first'
         .sort_values(by='de_dthr', ascending=True)
         .reset_index() # drop=True
         .rename(columns={'rct_no_payment':'buy_am'})   # 'cust_uniq_rct_nos':'rct_cnt'
)

#상반기의 recency는 2021.7.1을 기준으로 함
most_recent_date =pd.Timestamp("2021-07-01")
df_crm_first = (df_crm_first.groupby(['cust'])
          .agg({'de_dthr': lambda date: (most_recent_date - date.max()).days,
                'rct_no': lambda num: num.nunique(), 'buy_am':'sum'})
          .astype({'buy_am':'int'})
)
df_crm_first.columns = ['recency', 'frequency', 'monetary']
#          .reset_index()

# 5등급으로 나눔
df_crm_first["recency_score"] = pd.qcut(df_crm_first['recency'], 5, labels=[5, 4, 3, 2, 1])
df_crm_first["frequency_score"] = pd.qcut(df_crm_first['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
df_crm_first["monetary_score"] = pd.qcut(df_crm_first['monetary'], 5, labels=[1, 2, 3, 4, 5])

#datatype 형변환
df_crm_first[['recency_score','frequency_score','monetary_score']] = df_crm_first[['recency_score','frequency_score','monetary_score']].astype(int)

df_crm_first['rfm_score'] = ((0.3*df_crm_first['recency_score'] + 0.2*df_crm_first['frequency_score'] + 0.5*df_crm_first['monetary_score']) * 100 / 5).astype(int)

#기타:0-39점, 일반:40-59점, 우수:60-79점, 최우수:80-100점
df_crm_first["rfm_level"] = pd.cut(df_crm_first["rfm_score"], bins=4, right=False, labels=['기타고객','일반고객','우수고객','최우수고객'])

### 하반기

In [31]:
# 상반기 장바구니별로 groupby
df_crm_second = (df_second_half.groupby(['rct_no'])
         .agg({'cust':'first', 'de_dthr':'first', 'rct_no_payment':'first'}) # cust_uniq_rct_nos':'first'
         .sort_values(by='de_dthr', ascending=True)
         .reset_index() # drop=True
         .rename(columns={'rct_no_payment':'buy_am'})   # 'cust_uniq_rct_nos':'rct_cnt'
)

#하반기의 recency는 2022.1.1을 기준으로 함
most_recent_date =pd.Timestamp("2022-01-01")
df_crm_second = (df_crm_second.groupby(['cust'])
          .agg({'de_dthr': lambda date: (most_recent_date - date.max()).days,
                'rct_no': lambda num: num.nunique(), 'buy_am':'sum'})
          .astype({'buy_am':'int'})
)
df_crm_second.columns = ['recency', 'frequency', 'monetary']
#          .reset_index()

# 5등급으로 나눔
df_crm_second["recency_score"] = pd.qcut(df_crm_second['recency'], 5, labels=[5, 4, 3, 2, 1])
df_crm_second["frequency_score"] = pd.qcut(df_crm_second['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
df_crm_second["monetary_score"] = pd.qcut(df_crm_second['monetary'], 5, labels=[1, 2, 3, 4, 5])

#datatype 형변환
df_crm_second[['recency_score','frequency_score','monetary_score']] = df_crm_second[['recency_score','frequency_score','monetary_score']].astype(int)

df_crm_second['rfm_score'] = ((0.3*df_crm_second['recency_score'] + 0.2*df_crm_second['frequency_score'] + 0.5*df_crm_second['monetary_score']) * 100 / 5).astype(int)

#기타:0-39점, 일반:40-59점, 우수:60-79점, 최우수:80-100점
df_crm_second["rfm_level"] = pd.cut(df_crm_second["rfm_score"], bins=4, right=False, labels=['기타고객','일반고객','우수고객','최우수고객'])

### 상반기와 하반기 join

In [32]:
df_join = pd.merge(df_crm_first, df_crm_second, left_index=True, right_index=True, how='left') #.fillna('비활동고객')
df_rank = df_join[['rfm_level_x','rfm_level_y']].rename(columns={'rfm_level_x':'grade_first', 'rfm_level_y':'grade_second'})

df_rank['grade_second'] = df_rank['grade_second'].cat.add_categories('비활동고객').fillna('비활동고객')

# NCF 데이터 준비

## 유저

In [35]:
df_user = pd.merge(df_demo.set_index('cust'), df_rank, left_index=True, right_index=True)

In [37]:
df_user['label'] = df_user['grade_first'].str.cat(df_user['grade_second'], sep='/')

### 상품 구매 정보 벡터

상반기/하반기 구분하지 않기로 함

In [57]:
def get_pdde_ext_summary(df, prefix=''):
    '''
        df: 상품 구매 정보 DataFrame
        prefix: 변환 결과 컬럼에 붙일 prefix
        
        구매의 경우는 RFM 에 총 구매 빈도와, 구매 금액 랭크가 반영되어 있으므로...
        유통사 코드에 대한 빈도만 추가한다.
    '''
    zz = (df.groupby('cust').cop_c.value_counts(normalize=False)
       .unstack()
       .fillna(0.0)
       .rename_axis(columns=None)
       .add_prefix(f'{prefix}pdde_').add_suffix('_rank')
      )
    return pd.concat((zz[col].rank(pct=True, method='min') for col in zz.columns), axis=1) 

In [59]:
df_pdde_summary = get_pdde_ext_summary(df_pdde_raw)
df_merged = df_user.merge(df_pdde_summary, left_index=True, right_index=True, how='left')

for col in df_pdde_summary.columns:
    df_merged[col].fillna(0.0, inplace=True)

### 제휴사 이용 정보 벡터

상반기/하반기 구분하지 않기로 함

In [61]:
df_cop_u_raw = pd.read_csv(os.path.join(base_path, file_path_dic['cop_u']), dtype={'br_c':str, 'de_dt':str, 'de_hr':str})
df_cop_u = pd.concat((df_cop_u_raw[['cust', 'rct_no', 'cop_c', 'br_c', 'buy_am']], 
           pd.to_datetime(df_cop_u_raw.de_dt.str.cat(df_cop_u_raw.de_hr), format='%Y%m%d%H').rename('buy_dt')), axis=1).set_index('buy_dt').sort_index()

In [62]:
df_cop_u_first = df_cop_u.query("buy_dt < @half")
df_cop_u_second = df_cop_u.query("buy_dt >= @half")

제휴사 전체 이용 빈도/제휴사 개별 이용 빈도 등수, 금액 등수

In [63]:
def get_cop_summary(df, prefix=''):
    '''
        df: 제휴사 이용 정보 DataFrame
        prefix: 변환 결과 컬럼에 붙일 prefix
    '''
    yy = df.groupby('cust').agg({'cop_c':'count', 'buy_am':'sum'})
    zz = (df.groupby('cust').cop_c.value_counts(normalize=False)
       .unstack()
       .fillna(0.0)
       .rename_axis(columns=None)
       .add_prefix(f'{prefix}cop_').add_suffix('_rank')
      )
    return pd.concat((
        yy.cop_c.rank(pct=True, method='min').rename(f'{prefix}cop_use_rank'),
        yy.buy_am.rank(pct=True, method='min').rename(f'{prefix}cop_buy_rank'),
        pd.concat((zz[col].rank(pct=True, method='min') for col in zz.columns), axis=1) 
        ), axis=1)

제휴사 이용정보를 merge
- user left join cop
- NaN은 0.0 으로 채움

주의
- 상품구매: 25,035 명
- 제휴: 21,769 명
- 상품구매, 제휴 교집합: 18,062 명

In [64]:
df_cop_summary = get_cop_summary(df_cop_u)
df_merged = df_merged.merge(df_cop_summary, left_index=True, right_index=True, how='left')

for col in df_cop_summary.columns:
    df_merged[col].fillna(0.0, inplace=True)

### 엘페이 이용 정보 벡터

상반기/하반기 구분하지 않기로 함

In [65]:
df_lpay_raw = pd.read_csv(os.path.join(base_path, file_path_dic['lpay']), dtype={'de_dt':str, 'de_hr':str})
df_lpay = pd.concat((df_lpay_raw[['cust', 'rct_no', 'chnl_dv', 'cop_c', 'buy_am']], 
           pd.to_datetime(df_lpay_raw.de_dt.str.cat(df_lpay_raw.de_hr), format='%Y%m%d%H').rename('buy_dt')), axis=1).set_index('buy_dt').sort_index()

In [66]:
df_lpay_first = df_lpay.query("buy_dt < @half")
df_lpay_second = df_lpay.query("buy_dt >= @half")

엘페이 전체 이용 빈도/제휴사 개별 이용 빈도 등수, 금액 등수

In [67]:
def get_lpay_summary(df, prefix=''):
    '''
        df: cop_c 이용 정보 DataFrame
        prefix: 변환 결과 컬럼에 붙일 prefix
    '''
    yy = df.groupby('cust').agg({'cop_c':'count', 'buy_am':'sum'})
    zz = (df.groupby('cust').cop_c.value_counts(normalize=False)
       .unstack()
       .fillna(0.0)
       .rename_axis(columns=None)
       .add_prefix(f'{prefix}lpay_').add_suffix('_rank')
      )
    return pd.concat((
        yy.cop_c.rank(pct=True, method='min').rename(f'{prefix}lpay_use_rank'),
        yy.buy_am.rank(pct=True, method='min').rename(f'{prefix}lpay_buy_rank'),
        pd.concat((zz[col].rank(pct=True, method='min') for col in zz.columns), axis=1) 
        ), axis=1)

제휴사 이용정보를 merge
- user left join cop
- NaN은 0.0 으로 채움

주의
- 상품구매: 25,035 명
- 엘페이: 8,906 명
- 상품구매, 엘페이 교집합: 8,393 명

In [68]:
df_lpay_summary = get_lpay_summary(df_lpay)

In [69]:
df_merged = df_merged.merge(df_lpay_summary, left_index=True, right_index=True, how='left')

for col in df_lpay_summary.columns:
    df_merged[col].fillna(0.0, inplace=True)

In [91]:
df_user_merged = df_merged.copy()

## 아이템

In [178]:
df_pd_clac = pd.read_csv(os.path.join(base_path, file_path_dic['pd_clac'])).set_index('pd_c')

In [179]:
df_pd_clac

,pd_nm,clac_hlv_nm,clac_mcls_nm
pd_c,,,
PD0001,소파,가구,거실가구
PD0002,스툴/리빙의자,가구,거실가구
PD0003,탁자,가구,거실가구
PD0004,장식장/진열장,가구,거실가구
PD0005,기타가구,가구,기타가구
...,...,...,...
PD1929,여성향수,화장품/뷰티케어,향수
PD1930,남성향수,화장품/뷰티케어,향수
PD1931,남녀공용향수,화장품/뷰티케어,향수


In [241]:
df_pdde_raw.groupby('pd_c').cop_c.value_counts(normalize=False)

pd_c    cop_c
PD0001  A01      298
        A06       14
        A02        1
PD0002  A02      174
        A06       13
                ... 
PD1931  A02        8
        A03        2
PD1932  A01       45
PD1933  A01       36
        A06       14
Name: cop_c, Length: 5847, dtype: int64

In [181]:
zz = df_pdde_raw.groupby('pd_c').cop_c.value_counts(normalize=False).unstack().fillna(0.0).rename_axis(columns=None).add_suffix('_rank')

In [182]:
df_item_summary_cop_c = pd.concat((zz[col].rank(pct=True, method='min') for col in zz.columns), axis=1)

In [183]:
df_item_summary_cop_c

,A01_rank,A02_rank,A03_rank,A04_rank,A05_rank,A06_rank
pd_c,,,,,,
PD0001,0.827212,0.232281,0.000517,0.000517,0.000517,0.623383
PD0002,0.323332,0.588205,0.000517,0.000517,0.000517,0.605794
PD0003,0.543714,0.447491,0.000517,0.000517,0.000517,0.728401
PD0004,0.371961,0.463011,0.000517,0.000517,0.000517,0.659079
PD0005,0.892395,0.416451,0.000517,0.000517,0.000517,0.000517
...,...,...,...,...,...,...
PD1929,0.947232,0.317641,0.000517,0.814796,0.000517,0.948784
PD1930,0.792550,0.257113,0.000517,0.774961,0.000517,0.709260
PD1931,0.914123,0.310398,0.583032,0.000517,0.000517,0.892913


In [184]:
df_item_merged = pd.merge(df_pd_clac['clac_hlv_nm'], df_item_summary_cop_c, left_index=True, right_index=True)

In [185]:
df_item_merged

,clac_hlv_nm,A01_rank,A02_rank,A03_rank,A04_rank,A05_rank,A06_rank
pd_c,,,,,,,
PD0001,가구,0.827212,0.232281,0.000517,0.000517,0.000517,0.623383
PD0002,가구,0.323332,0.588205,0.000517,0.000517,0.000517,0.605794
PD0003,가구,0.543714,0.447491,0.000517,0.000517,0.000517,0.728401
PD0004,가구,0.371961,0.463011,0.000517,0.000517,0.000517,0.659079
PD0005,가구,0.892395,0.416451,0.000517,0.000517,0.000517,0.000517
...,...,...,...,...,...,...,...
PD1929,화장품/뷰티케어,0.947232,0.317641,0.000517,0.814796,0.000517,0.948784
PD1930,화장품/뷰티케어,0.792550,0.257113,0.000517,0.774961,0.000517,0.709260
PD1931,화장품/뷰티케어,0.914123,0.310398,0.583032,0.000517,0.000517,0.892913


In [186]:
df_user_merged.head(3)

,ma_fem_dv,ages,zon_hlv,grade_first,grade_second,label,pdde_A01_rank,pdde_A02_rank,pdde_A03_rank,pdde_A04_rank,...,lpay_A05_rank,lpay_A06_rank,lpay_B01_rank,lpay_C01_rank,lpay_C02_rank,lpay_D01_rank,lpay_D02_rank,lpay_E01_rank,lpay_L00_rank,lpay_L01_rank
cust,,,,,,,,,,,,,,,,,,,,,
M000034966,여성,40대,Z07,일반고객,일반고객,일반고객/일반고객,0.388453,0.628153,0.000037,0.715830,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
M000136117,여성,30대,Z11,최우수고객,최우수고객,최우수고객/최우수고객,0.928447,0.452019,0.000037,0.000037,...,0.000112,0.478554,0.000112,0.000112,0.000112,0.719066,0.000112,0.000112,0.000112,0.000112
M000225114,여성,40대,Z17,일반고객,최우수고객,일반고객/최우수고객,0.971877,0.000037,0.684735,0.000037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## 변환

In [94]:
from sklearn.preprocessing import OneHotEncoder

### 아이템

In [187]:
df_item_merged = pd.concat((df_item_merged, pd.Series(list(range(0, len(df_item_merged.index))), index=df_item_merged.index, name='pd_c_no')
                           ), axis=1)

In [189]:
item_category_features = ['clac_hlv_nm']
item_continuous_features = ['A01_rank', 'A02_rank', 'A03_rank', 'A04_rank',
       'A05_rank', 'A06_rank', 'pd_c_no']

In [192]:
item_category_enc = OneHotEncoder()

item_category_enc.fit(df_item_merged[item_category_features])

itemX = item_category_enc.transform(df_item_merged[item_category_features]).toarray()

In [242]:
item_category_enc.categories_

[array(['가구', '건강식품', '건강용품', '건해산물', '계절가전', '공구/안전용품', '과일', '과자',
        '구기/필드스포츠', '금융/보험서비스', '기타(비상품)', '기타상품', '남성의류', '냉동식품',
        '냉장/세탁가전', '냉장식품', '담배', '대용식', '모바일', '문구/사무용품', '병통조림', '상품권',
        '생활/렌탈서비스', '생활/주방가전', '서적/음반/악기', '세제/위생', '속옷/양말/홈웨어', '수산물',
        '스포츠패션', '시즌스포츠', '식기/조리기구', '아웃도어/레저', '양곡', '여성의류', '여행/레저서비스',
        '영상/음향가전', '완구', '원예/애완', '유아동의류', '유아식품', '유제품', '음료', '인테리어/조명',
        '자동차용품', '조리식품', '조미료', '주류', '주방잡화', '채소', '청소/세탁/욕실용품', '축산물',
        '출산/육아용품', '침구/수예', '커피/차', '컴퓨터', '테넌트/음식점', '패션잡화', '퍼스널케어',
        '헬스/피트니스', '화장품/뷰티케어'], dtype=object)]

벡터 설명
- 0~59: one-hot encoding vector 모음
- 60~65: continuous vector
- 66: pd_c 인덱스값

In [223]:
df_np_item = pd.DataFrame(np.hstack((itemX, df_item_merged[item_continuous_features].values)), index= df_item_merged.index)

In [226]:
df_np_item.to_csv('csv_np_item.csv')

### 사용자

In [195]:
df_user_merged = pd.concat((df_user_merged, pd.Series(list(range(0, len(df_user_merged.index))), index=df_user_merged.index, name='cust_no')
                           ), axis=1)

In [197]:
user_category_features = ['ma_fem_dv', 'ages', 'grade_first', 'grade_second']

user_continuous_feature = ['pdde_A01_rank', 'pdde_A02_rank', 'pdde_A03_rank', 'pdde_A04_rank',
       'pdde_A05_rank', 'pdde_A06_rank', 'cop_use_rank', 'cop_buy_rank',
       'cop_B01_rank', 'cop_C01_rank', 'cop_C02_rank', 'cop_D01_rank',
       'cop_D02_rank', 'cop_E01_rank', 'lpay_use_rank', 'lpay_buy_rank',
       'lpay_A01_rank', 'lpay_A02_rank', 'lpay_A03_rank', 'lpay_A04_rank',
       'lpay_A05_rank', 'lpay_A06_rank', 'lpay_B01_rank', 'lpay_C01_rank',
       'lpay_C02_rank', 'lpay_D01_rank', 'lpay_D02_rank', 'lpay_E01_rank',
       'lpay_L00_rank', 'lpay_L01_rank', 'cust_no']

In [200]:
user_category_enc = OneHotEncoder()

user_category_enc.fit(df_user_merged[user_category_features])

userX = user_category_enc.transform(df_user_merged[user_category_features]).toarray()

In [243]:
user_category_enc.categories_

[array(['남성', '여성'], dtype=object),
 array(['20대', '30대', '40대', '50대', '60대', '70대'], dtype=object),
 array(['기타고객', '우수고객', '일반고객', '최우수고객'], dtype=object),
 array(['기타고객', '비활동고객', '우수고객', '일반고객', '최우수고객'], dtype=object)]

벡터 설명
- 0~16: one-hot encoding vector 모음
- 17~46: continuous vector
- 47: cust 인덱스값

In [224]:
df_np_user = pd.DataFrame(np.hstack((userX, df_user_merged[user_continuous_feature].values)), index= df_user_merged.index)

In [227]:
df_np_user.to_csv('csv_np_user.csv')

### 사용자-구매 데이터

### 사용자, 아이템 인덱스 사전

In [228]:
df_cust_pd_c = pd.merge(pd.merge(df_pdde_raw[['cust', 'pd_c']], df_item_merged[['pd_c_no']].reset_index(), on = 'pd_c'),
         df_user_merged[['cust_no']].reset_index(), on = 'cust')

한번만 구매했으면 implicit feedback '1' 로 처리하기로 함 (다수 구매를 한번으로 처리)

cust, pd_c, pd_c_no, cust_no 담고 있는 사전

In [229]:
df_cust_pd_c_compact = df_cust_pd_c.groupby(['cust','pd_c'], as_index=False).agg({'pd_c_no':'first', 'cust_no':'first'})

In [231]:
df_cust_pd_c_compact.to_csv('csv_user_item_dic.csv', index=False)

In [240]:
df_cust_pd_c_compact

,cust,pd_c,pd_c_no,cust_no
0,M000034966,PD0116,115,0
1,M000034966,PD0169,168,0
2,M000034966,PD0178,177,0
3,M000034966,PD0204,203,0
4,M000034966,PD0218,217,0
...,...,...,...,...
1615186,M999962961,PD1857,1856,25034
1615187,M999962961,PD1889,1888,25034
1615188,M999962961,PD1896,1895,25034
1615189,M999962961,PD1918,1917,25034
